<a href="https://colab.research.google.com/github/keiohta/2020_deeprl_summer_school_lecture3/blob/master/rlss2020_lecture3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 強化学習サマースクール 第3回 Model-based RL

## 目次
1. [はじめに]()
1. [モデル予測制御]()
1. [アンサンブルモデルによるモデルベース強化学習]()

## はじめに
第3回では、モデルベースRLとしてダイナミクス（状態遷移）モデルを用いた制御手法について取り扱います。
本資料では、[Pendulum](https://gym.openai.com/envs/Pendulum-v0/)を題材に、基礎的なモデルベースRLのコードを読み書きすることで理論と実装両面の理解を進めることを目的とします。

まず初めに、今回の演習で必要なライブラリを一度にインストールしておきましょう。


In [ ]:
!apt update > /dev/null 2>&1
!apt install xvfb > /dev/null 2>&1
!pip install gym-notebook-wrapper cpprb tf2rl > /dev/null 2>&1

## モデル予測制御（MPC）

MPCと一言にまとめてもたくさん種類がありますが、本演習では下記論文で提案されているものを考えます。
- [Neural Network Dynamics for Model-Based Deep Reinforcement Learning with Model-Free Fine-Tuning](https://arxiv.org/abs/1708.02596)

上記論文では、ダイナミクスモデルを学習し、それを用いてRandom-sample Shooting (RS) という方法を用いて制御問題を解きます。

### RS: Random-sample Shooting
RSでは、ダイナミクスモデル $f$ と報酬（またはコスト）関数 $g$ が（正確でないにせよ）既知であると仮定し、
それを用いて現在の状態 $s_t \in \mathbb{R}^{N^{state}}$ における行動 $a_t$ を生成します。

ダイナミクスモデル $f$ が既知なので、次の状態を $s_{t+1} = f(s_t, a_t)$ のように予測することができます。
更に、報酬関数を用いて $r_{t+1} = g(s_t, a_t, s_{t+1})$ からその状態において実行した行動がどれくらい良いかが得られます。
このように、次の状態・報酬を反復的に $M$ 回計算することで、 $M$ ステップ後までの累積報酬和 $R_t^{t+M} = \sum_{m=1}^M r_{t+m}$が得られます。

RSでは、このアイデアを用いて**ランダムに**行動を $N^\text{episode}$ エピソード分生成し、その中で最も
累積報酬和が高かった最初の行動を選択し、1ステップ分だけエピソードを進めます。
具体的には、以下のように進めます。

1. 現在の状態を $s_t$ とし、$N^\text{episode}$エピソード分を複製する $\mathbf{s} \in \mathbb{R}^{N^\text{episode} \times N^{state}}$ 
1. $N^\text{episode}$ エピソード分のランダムな行動 $\mathbf{a} \in \mathbb{R}^{N^\text{episode} \times N^{action}}$ を生成する
1. ダイナミクスモデルを用いて次の状態と報酬を予測する
1. $T$ステップ先読みするまで2に戻って再度繰り返す
1. $N^\text{episode}$ エピソードの中で最も累積報酬が高いエピソードを選択し、その最初の行動を用いて1ステップ環境を進める

### 全体の流れ
今回扱う手法では、ダイナミクスモデルを学習しながらRSを行います。
全体的な流れは以下のようになります。

1. ランダムに遷移を収集し状態遷移モデル $f$ を事前学習
1. $N^\text{episode}$ エピソード分繰り返し
  1. エピソード終端条件を満たすまで繰り返し
    1. RSを用いて環境を進める
    1. 得られた遷移 $(s_t, a_t, s_{t+1})$ をバッファ $\mathcal{D}$ に保存
  1. ダイナミクスモデル $f$ を学習

それでは実装していきましょう。

#### 環境の実装

まず初めに今回の演習で使う環境であるOpenAI Gymの[Pendulum](https://gym.openai.com/envs/Pendulum-v0/)環境を用意します。

Pendulumは状態数が3（現在の振り子の角度と角速度：$\{\sin\theta, \cos\theta, \dot{\theta}\}$）, 行動数が1（トルク：$\tau$）の連続値（非離散値）入出力の中で最も簡単な環境の一つです。

また、Pendulumがどのような動作をするかを可視化して確認しておきましょう。
制御器を設計する前に環境の特徴を把握することは非常に重要です。

In [ ]:
import gnwrapper
import gym
 
env = gym.make("Pendulum-v0")

# 可視化用の環境。JupyterNotebookで可視化するためのラッパーをかましています
monitor_env = gnwrapper.Monitor(gym.make("Pendulum-v0"), size=(400, 300), directory='.', force=True,
                                video_callable=lambda ep: True)
episode_max_steps = 200

for episode_idx in range(3):
    monitor_env.reset()
    total_rew = 0.
    for _ in range(episode_max_steps):
        act = monitor_env.action_space.sample()
        _, rew, done, _ = monitor_env.step(act)
        total_rew += rew
        if done:
            break
    print("iter={0: 3d} total reward: {1: 4.4f}".format(episode_idx, total_rew))

monitor_env.display()

iter=  0 total reward: -975.4136
iter=  1 total reward: -1310.4021
iter=  2 total reward: -785.9991


'openaigym.video.0.103.video000000.mp4'

'openaigym.video.0.103.video000001.mp4'

また、ここでPendulum特有の報酬関数についても実装しておきましょう。
報酬計算部分のコードを[ここ](https://github.com/openai/gym/blob/6df1b994bae791667a556e193d2a215b8a1e397a/gym/envs/classic_control/pendulum.py#L51)から抜粋して使用します。
基本的には、以下のような報酬関数になっています。

$
r = -\left( \left\| \theta_t - \theta^\text{goal}  \right\|^{2} + 0.1 \times \dot{\theta}_t^2 + 0.001 \times \tau_t^2 \right)
$

大まかには、目標角度 $\theta^\text{goal}$ に到達する際に、角速度と必要なトルクを最小化するような方策が最適な方策であることが分かります。
また、**0に近いほど良い**ことも分かります。

In [ ]:
def angle_normalize(x):
    return ((x + np.pi) % (2 * np.pi)) - np.pi
 
 
def reward_fn(obses, acts):
    is_single_input = obses.ndim == acts.ndim and acts.ndim == 1
    if is_single_input:
        thetas = np.arctan2(obses[1], obses[0])
        theta_dots = obses[2]
    else:
        assert obses.ndim == acts.ndim == 2
        assert obses.shape[0] == acts.shape[0]
        acts = np.squeeze(acts)
        thetas = np.arctan2(obses[:, 1], obses[:, 0])
        theta_dots = obses[:, 2]
        assert thetas.shape == theta_dots.shape == acts.shape

    acts = np.clip(acts, -2, 2)
    costs = angle_normalize(thetas) ** 2 + .1 * theta_dots ** 2 + .001 * (acts ** 2)

    return -costs

#### ダイナミクスモデルの実装

続いて、オンラインで学習するダイナミクスモデル $f_\theta$ を実装します。
ダイナミクスモデルはどのような関数近似器で近似しても良いですが、今回は2層のMLP (Multi-layer Perceptron) を用いましょう。

一点注意が必要なのは、今回再現実装しようとしている[論文](https://arxiv.org/abs/1708.02596)では、直接次の状態を予測するのではなく、**次の状態と現在の状態との差分**を予測します。つまり、下記損失関数を最小化するようなパラメータ $\theta$ を学習します。

$
\mathcal{E}(\theta)=\frac{1}{|\mathcal{D}|} \sum_{\left(\mathbf{s}_{t}, \mathbf{a}_{t}, \mathbf{s}_{t+1}\right) \in \mathcal{D}} \frac{1}{2}\left\|\left(\mathbf{s}_{t+1}-\mathbf{s}_{t}\right)-f_{\theta}\left(\mathbf{s}_{t}, \mathbf{a}_{t}\right)\right\|^{2}
$

ここで、$\mathcal{D}$ は遷移を保存したバッファとなります。
このような次の状態を直接予測するのではなく、その差分だけを予測する手法はよく使われるテクニックです。
これにより出力の分散をある程度抑えられるなどのメリットがあります。

それでは、実装していきましょう。

In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import Dense

class DynamicsModel(tf.keras.Model):
    def __init__(self, input_dim, output_dim, units=(32, 32)):
        super().__init__(name="MLP")

        self.l1 = Dense(units[0], name="L1", activation="relu")
        self.l2 = Dense(units[1], name="L2", activation="relu")
        self.l3 = Dense(output_dim, name="L3", activation="linear")

        self._optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        with tf.device("/gpu:0"):
            self(tf.constant(np.zeros(shape=(1, input_dim), dtype=np.float32)))

    @tf.function
    def call(self, inputs):
        features = self.l1(inputs)
        features = self.l2(features)
        return self.l3(features)

    def predict(self, inputs):
        assert isinstance(inputs, np.ndarray)
        assert inputs.ndim == 2

        with tf.device("/gpu:0"):
            outputs = self.call(inputs)

        return outputs.numpy()

    @tf.function
    def fit(self, inputs, labels):
        with tf.GradientTape() as tape:
            predicts = self(inputs)
            loss = 0.5 * tf.reduce_mean(tf.square(labels - predicts))
        grads = tape.gradient(loss, self.trainable_variables)
        self._optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return loss

obs_dim = env.observation_space.high.size
act_dim = env.action_space.high.size

dynamics_model = DynamicsModel(input_dim=obs_dim + act_dim, output_dim=obs_dim)


def predict_next_state(obses, acts):
    assert obses.shape[0] == acts.shape[0]
    obs_diffs = dynamics_model.predict(np.concatenate([obses, acts], axis=1))
    assert obses.shape == obs_diffs.shape
    next_obses = obses + obs_diffs
    return next_obses

また、ダイナミクスモデルを学習する際に遷移を保存しておくバッファを実装します。
リングバッファを自分で実装しても良いですが、今回は既存のライブラリ（[cpprb](https://github.com/ymd-h/cpprb)）を用います。
バッファサイズは10Kとしてみます。


In [ ]:
from cpprb import ReplayBuffer

rb_dict = {
    "size": 10000,
    "default_dtype": np.float32,
    "env_dict": {
        "obs": {
            "shape": env.observation_space.shape},
        "next_obs": {
            "shape": env.observation_space.shape},
        "act": {
            "shape": env.action_space.shape}}}

dynamics_buffer = ReplayBuffer(**rb_dict)

#### RS (Random-sample Shooting) の実装

次にRSを実装します。便利なので方策を用意しましょう。

In [ ]:
class RandomPolicy:
    def __init__(self, max_action, act_dim):
        self._max_action = max_action  # action の最大値
        self._act_dim = act_dim  # action の次元数

    def get_actions(self, batch_size):
        # 一様分布からバッチサイズ分ランダムにサンプリング
        return np.random.uniform(
            low=-self._max_action,
            high=self._max_action,
            size=(batch_size, self._act_dim))

policy = RandomPolicy(
    max_action=env.action_space.high[0],
    act_dim=env.action_space.high.size)


def random_shooting(init_obs, n_mpc_episodes=64, horizon=20):
    init_actions = policy.get_actions(batch_size=n_mpc_episodes)
    total_rewards = np.zeros(shape=(n_mpc_episodes,))
    obses = np.tile(init_obs, (n_mpc_episodes, 1))

    for i in range(horizon):
        acts = init_actions if i == 0 else policy.get_actions(batch_size=n_mpc_episodes)
        next_obses = predict_next_state(obses, acts)
        rewards = reward_fn(obses, acts)
        total_rewards += rewards
        obses = next_obses

    idx = np.argmax(total_rewards)
    return init_actions[idx]

#### 実行スクリプト

さて、いよいよRSを実行するコードを実装します。
冒頭の「全体の流れ」をコメントで繰り返しながら記述していきます。

In [ ]:
batch_size = 100
n_episodes = 100


# ランダムに遷移を収集し状態遷移モデルを収集します
for _ in range(10):
    obs = env.reset()
    for _ in range(200):
        act = env.action_space.sample()
        next_obs, _, done, _ = env.step(act)
        dynamics_buffer.add(obs=obs, act=act, next_obs=next_obs)
        obs = next_obs
        if done:
            break


def fit_dynamics(n_iter=50):
    mean_loss = 0.
    for _ in range(n_iter):
        samples = dynamics_buffer.sample(batch_size)
        inputs = np.concatenate([samples["obs"], samples["act"]], axis=1)
        labels = samples["next_obs"] - samples["obs"]
        mean_loss += dynamics_model.fit(inputs, labels).numpy()
    return mean_loss

# ダイナミクスモデルを事前学習します
fit_dynamics(n_iter=1000)


total_steps = 0
for episode_idx in range(n_episodes):
    total_rew = 0.

    obs = env.reset()
    for _ in range(episode_max_steps):
        total_steps += 1
        act = random_shooting(obs)
        next_obs, rew, done, _ = env.step(act)
        dynamics_buffer.add(
            obs=obs, act=act, next_obs=next_obs)
        total_rew += rew
        if done:
            break
        obs = next_obs

    mean_loss = fit_dynamics(n_iter=100)
    if episode_idx % 5 == 0:
        print("iter={0: 3d} total reward: {1: 4.4f} mean loss: {2:.6f}".format(episode_idx, total_rew, mean_loss))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

iter=  0 total reward: -253.4273 mean loss: 0.015259
iter=  5 total reward: -592.7901 mean loss: 0.009036
iter= 10 total reward: -609.1725 mean loss: 0.006151
iter= 15 total reward: -1.3522 mean loss: 0.004515
iter= 20 total reward: -1.3761 mean loss: 0.003423
iter= 25 total reward: -585.1795 mean loss: 0.003186
iter= 30 total reward: -1.9137 mean loss: 0.002517
iter= 35 total reward: -1.4302 mean loss: 0.002224
iter= 40 total reward: -2.2567 mean loss: 0.002041
iter= 45 total reward: -359.3084 mean los

最後に、学習済みモデルを使って振り子の動きを可視化してみましょう。

In [ ]:
# 学習済みモデルでの結果が見たいので過去のビデオリストを削除します
monitor_env.display(reset=True)

for episode_idx in range(3):
    obs = monitor_env.reset()
    total_rew = 0.
    for _ in range(episode_max_steps):
        act = random_shooting(obs)
        next_obs, rew, done, _ = monitor_env.step(act)
        total_rew += rew
        if done:
            break
        obs = next_obs
    print("iter={0: 3d} total reward: {1: 4.4f}".format(episode_idx, total_rew))

monitor_env.display()

'openaigym.video.0.103.video000000.mp4'

'openaigym.video.0.103.video000001.mp4'

iter=  0 total reward: -122.5595
iter=  1 total reward: -126.1413
iter=  2 total reward: -122.5665


'openaigym.video.0.103.video000002.mp4'

'openaigym.video.0.103.video000003.mp4'

'openaigym.video.0.103.video000004.mp4'

学習済みモデルで振り子の振り上げがきちんとできていることが確認できましたか？
ただ、ご覧のように性能がイマイチですね。RSは実装が容易で、少ないサンプル数で解けることがメリットとして挙げられますが、例えば次のような欠点があります。

- $M$ステップ分の累積報酬和が最も良かったエピソードの**最初の**行動を選択するが、それが良いとは限らない（最初のステップは全然ダメでも2ステップ目以降が良い場合も考えられます）
- 先読みステップ数、エピソード数の設計にドメイン知識が必要

## モデルアンサンブルによるモデル誤差軽減
モデルベースRLの利点・欠点をいくつかまとめましょう。
- 利点
  - サンプル効率が良い（サンプル効率は同じ性能に達するために必要な環境との相互作用数を指し、少ない方が良いです）
  - ダイナミクスモデルがわかっているので既存のプラニングアルゴリズムが使える
- 欠点
  - モデル誤差が大きい（ダイナミクスモデルが十分に実際の環境を近似できていない）と性能が出ない

2つ目の演習では、この欠点に挑戦します。
モデル誤差低減の方法はいくつかありますが、ここでは複数のモデルを同時に用いる手法により解決を試みます。
より具体的には、関数近似器を複数用意し、それらを効果的に用いることで方策の性能を向上させます。

### ME-TRPO

本演習では、下記論文について取り扱います。
- [Model-Ensemble Trust-Region Policy Optimization](https://arxiv.org/abs/1802.10592)

ME-TRPOでは、学習したダイナミクスモデルを**強化学習**に用います。
具体的には、ダイナミクスモデルで**擬似的な**遷移を生成し、それを用いて方策と価値関数を最適化します。

### 全体の流れ
以下のようなアルゴリズムになります。

1. 方策 $\pi_\theta$、ダイナミクスモデル $f_\phi$ の初期化
1. repeat
    1. 実環境 $f$ で $\pi_\theta$ を使ってサンプル収集
    1. ダイナミクスモデル $f_\phi$ の更新
    1. repeat
        1. $\pi_\theta$ と $f_\phi$ を使って遷移データを生成
        1. 生成された遷移データを使って $\pi_\theta$ を更新
        1. 方策評価 $\eta(\theta; \phi)$
    1. until 方策評価が高い限り
1. until 終了条件を満たすまで

### 準備
環境の設定をします。

In [ ]:
episode_max_steps = 100
debug = False

ダイナミクスモデルを**複数**定義します。
また、ダイナミクスモデル学習用のバッファを初期化しておきます。

In [ ]:
# ダイナミクスモデルを5つ用います
n_dynamics_model = 5

obs_dim = env.observation_space.high.size
act_dim = env.action_space.high.size

dynamics_models = [DynamicsModel(
    input_dim=obs_dim + act_dim, output_dim=obs_dim) for _ in range(n_dynamics_model)]


def predict_next_state(obses, acts, idx=None):
    is_single_input = obses.ndim == acts.ndim and acts.ndim == 1
    if is_single_input:
        obses = np.expand_dims(obses, axis=0)
        acts = np.expand_dims(acts, axis=0)

    inputs = np.concatenate([obses, acts], axis=1)
    idx = np.random.randint(n_dynamics_model) if idx is None else idx
    obs_diffs = dynamics_models[idx].predict(inputs)

    if is_single_input:
        return obses[0] + obs_diffs
    return obses + obs_diffs

方策を用意します。[論文](https://arxiv.org/abs/1802.10592)では[TRPO](https://arxiv.org/abs/1502.05477)を用いましたが、やや複雑なので代わりに[PPO](https://arxiv.org/abs/1707.06347)を用います。

In [ ]:
from tf2rl.algos.ppo import PPO

policy = PPO(
    state_shape=env.observation_space.shape,
    action_dim=env.action_space.shape[0],
    max_action=env.action_space.high[0],
    is_discrete=False,
    batch_size=64,
    actor_units=(32, 32),
    critic_units=(32, 32),
    n_epoch=10,
    lr_actor=3e-4,
    lr_critic=3e-4,
    hidden_activation_actor="tanh",
    hidden_activation_critic="tanh",
    discount=0.9,
    lam=0.95,
    entropy_coef=0.,
    horizon=2048,
    normalize_adv=True,
    enable_gae=True,
    gpu=0)

def clip_action(action):
    return np.clip(action, env.action_space.low, env.action_space.high)

また、方策学習用のデータを格納するバッファを用意します。

In [ ]:
rb_dict = {
    "size": policy.horizon,
    "default_dtype": np.float32,
    "env_dict": {
        "obs": {
            "shape": env.observation_space.shape},
        "act": {
            "shape": env.action_space.shape},
        "logp": {},
        "val": {},
        "done": {},
        "ret": {},
        "adv": {}}}
on_policy_buffer = ReplayBuffer(**rb_dict)
rb_dict["env_dict"].pop("adv")
tmp_on_policy_buffer = ReplayBuffer(**rb_dict)

### 実環境 $f$ での $\pi_\theta$ を使って遷移データを生成


In [ ]:
def collect_transitions_real_env():
    obs = env.reset()
    episode_steps = 0
    for _ in range(policy.horizon):
        episode_steps += 1
        act, _ = policy.get_action(obs)
        next_obs, *_ = env.step(clip_action(act))
        dynamics_buffer.add(obs=obs, act=act, next_obs=next_obs)
        obs = next_obs
        if episode_steps == episode_max_steps:
            episode_steps = 0
            obs = env.reset()

# dynamics_buffer.clear()
# collect_transitions_real_env()
# print(dynamics_buffer.get_stored_size())

2048


### ダイナミクスモデルの更新


In [ ]:
def fit_dynamics(n_iter=50):
    mean_losses = np.zeros(shape=(n_dynamics_model,), dtype=np.float32)
    for _ in range(n_iter):
        samples = dynamics_buffer.sample(batch_size)
        inputs = np.concatenate([samples["obs"], samples["act"]], axis=1)
        labels = samples["next_obs"] - samples["obs"]
        for i, dynamics_model in enumerate(dynamics_models):
            mean_losses[i] += dynamics_model.fit(inputs, labels).numpy()
    return mean_losses


0 [0.34488493 0.28642735 0.27012318 0.67826986 0.38406006]
1 [0.0918671  0.08610974 0.07559161 0.18137212 0.08200469]
2 [0.05403963 0.05462363 0.04753036 0.08090855 0.04929909]
3 [0.03931221 0.04292368 0.03827332 0.05412946 0.0387259 ]
4 [0.03053864 0.03423731 0.03009051 0.04161959 0.03084332]
5 [0.02661211 0.02909944 0.02561474 0.03495678 0.02474855]
6 [0.02283844 0.02557291 0.02337181 0.03130296 0.02336784]
7 [0.02118918 0.02297932 0.02109918 0.02785218 0.02015374]
8 [0.01867778 0.02129352 0.01953625 0.0251757  0.01775236]
9 [0.01731122 0.01948348 0.01803065 0.02313891 0.01659614]
10 [0.01556237 0.0178409  0.01706237 0.02034613 0.01418224]
11 [0.01449707 0.01738181 0.01620633 0.01974195 0.01380723]
12 [0.01311959 0.01592669 0.01487688 0.01839787 0.01263544]
13 [0.01299621 0.0149665  0.01386813 0.01735435 0.01128761]
14 [0.01130974 0.01360977 0.01252299 0.01583349 0.01038169]
15 [0.01138868 0.01319942 0.01216775 0.01572832 0.00987725]
16 [0.01036442 0.01291297 0.01189359 0.01455789 0.

### $\pi_\theta$ と $f_\phi$ を使って遷移データを生成


In [ ]:
def collect_transitions_sim_env():
    """
    Generate transitions using dynamics model
    :return:
    """
    on_policy_buffer.clear()
    n_episodes = 0
    ave_episode_return = 0
    while on_policy_buffer.get_stored_size() < policy.horizon:
        obs = env.reset()
        episode_return = 0.
        for _ in range(episode_max_steps):
            act, logp, val = policy.get_action_and_val(obs)
            env_act = clip_action(act)
            if debug:
                next_obs, rew, _, _ = env.step(env_act)
            else:
                next_obs = predict_next_state(obs, env_act)
                rew = reward_fn(obs, act)[0]
            tmp_on_policy_buffer.add(obs=obs, act=act, next_obs=next_obs, rew=rew,
                                     done=False, logp=logp, val=val)
            obs = next_obs
            episode_return += rew
        finish_horizon(last_val=val)
        ave_episode_return += episode_return
        n_episodes += 1
    return ave_episode_return / n_episodes


def finish_horizon(last_val=0):
    samples = tmp_on_policy_buffer._encode_sample(np.arange(tmp_on_policy_buffer.get_stored_size()))
    rews = np.append(samples["rew"], last_val)
    vals = np.append(samples["val"], last_val)

    # GAE-Lambda advantage calculation
    deltas = rews[:-1] + policy.discount * vals[1:] - vals[:-1]
    advs = discount_cumsum(deltas, policy.discount * policy.lam)

    # Rewards-to-go, to be targets for the value function
    rets = discount_cumsum(rews, policy.discount)[:-1]
    on_policy_buffer.add(
        obs=samples["obs"], act=samples["act"], done=samples["done"],
        ret=rets, adv=advs, logp=np.squeeze(samples["logp"]))
    tmp_on_policy_buffer.clear()


collect_transitions_sim_env()


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



KeyError: ignored